This notebook is used to store helper functions. Normally only used once.

# Import Libraries

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Prune Dataset


In [ ]:
##Read in all data from FSD50K dataset
dataset_path = '/content/drive/MyDrive/colab_notebooks/deep_learning/deep_learning_project/FSD50K'
filenames = pd.read_csv(os.path.join(dataset_path, 'filenames.csv'))['filename'].values
dev_path = os.path.join(dataset_path,'FSD50K.dev_audio')
eval_path = os.path.join(dataset_path,'FSD50K.eval_audio')
gt_dir = os.path.join(dataset_path, 'FSD50K.ground_truth')

df_vocab = pd.read_csv(os.path.join(gt_dir, 'vocabulary.csv'))
df_eval = pd.read_csv(os.path.join(gt_dir, 'eval.csv'))
df_dev = pd.read_csv(os.path.join(gt_dir, 'dev.csv'))

In [ ]:
df_eval['split'] = 'test'
df_data = pd.concat([df_dev, df_eval], ignore_index = True)
#Add column that keeps track of number of labels
df_data['len_cat'] = df_data['labels'].apply(lambda x: len(x.split(',')))

In [ ]:
#The five classes we have chosen to keep
labels = ['Music','Human_voice', 'Animal', 'Vehicle','Water']

#Count by class (for all three sets)
count = 2100
test = int(2100*0.2)
val = int(2100*0.15)

df_slim = pd.DataFrame()
for label in labels:
    ##Index data from current label while checking there is no overlap with other labels
    other = "|".join([l for l in labels if l != label])
    df = df_data[(df_data['labels'].str.contains(label))&(~df_data['labels'].str.contains(other, regex=True))].sort_values(by='len_cat')
    df['dir_path'] = dev_path
    df.loc[df['split']=='test', 'dir_path'] = eval_path
    df = df.drop(columns=['mids', 'split', 'len_cat', 'labels'])
    df['label'] = label

    #check that filename is in local dir
    df['filename'] = df['fname'].apply(lambda x : str(x) + ".wav")
    df = df[df['filename'].isin(filenames)]
    df = df.drop(columns=['filename']).reset_index(drop=True)
    
    ##Take the rows with the lowest count of labels
    df = df[:count]
    
    ##Train test val split for category
    df = df.sample(frac = 1).reset_index(drop=True)
    df['split'] = 'train'
    df.loc[:test, 'split'] = 'test'
    df.loc[test:test+val, 'split'] = 'val'
    
    df_slim = pd.concat([df_slim, df], ignore_index=True)

df_slim

In [ ]:
##Save dataframe so we always have the same train/val/test split
df_slim = df_slim.sample(frac=1).reset_index(drop=True)
df_slim.to_csv(os.path.join(dataset_path, 'dataset_slim.csv'), index=False)

# Calculate Mean and Std for normalizing input data

In [ ]:
#Read in data
dataset_path = '/content/drive/MyDrive/colab_notebooks/deep_learning/deep_learning_project/FSD50K'
gt_dir = os.path.join(dataset_path, 'FSD50K.ground_truth')
df_data = pd.read_csv(os.path.join(dataset_path, 'dataset_slim.csv'))

#Create dictionary to map labels to indices in y
labels = sorted(list(set(df_data['label'].values)))
label_to_index = {l:i for l, i in zip(labels, range(len(labels)))}
index_to_label = dict((v,k) for k,v in label_to_index.items())

batch_size = 16
num_workers = 2

train_loader = torch.utils.data.DataLoader(
            FSD50Dataset(df_data, 'train', label_to_index),
            batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)


In [ ]:
mean = []
std = []

for _, (X, _) in tqdm(enumerate(train_loader)):
    mean.append(torch.mean(X))
    std.append(torch.std(X))

print(np.mean(mean), np.mean(std))


427it [35:15,  4.95s/it]

-3.3852494 4.416891
